Import needed library

In [ ]:
from github import Github
import csv
import datetime

Initialize things

In [ ]:
# Get access to GitHub
g = Github("'''Personal Access Token'''")

# Determine the destination file 
csv_file = 'raw_data.csv'

# Detect the time range
since_date = datetime.datetime(2023, 1, 1)
until_date = datetime.datetime(2024, 1, 1) 

Get data

In [ ]:
# Get the repository
repo = g.get_repo("dmlc/xgboost")
# Open the file in write mode
with open(csv_file, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Commit ID', 'Commit message', 'Contributor', 'Commit date', 'Files Changed', 'Author', 'Author Email', 'Passed all test'])
    # Get all commits in time range
    commits = repo.get_commits(since=since_date, until=until_date)
    # and then fill the data in proper position.
    for commit in commits:
        commit_id = commit.sha
        commit_message = commit.commit.message
        contributor = commit.author.login if commit.author else 'N/A'
        commit_date = commit.commit.author.date.strftime("%Y-%m-%d")
        files_changed = commit.files
        changed_files = [file.filename for file in files_changed]
        # Get the author information to make sure the author and contributor is one person.
        author_name = commit.commit.author.name
        author_email = commit.commit.author.email
        # Check if the commit passed all tests
        check_runs = commit.get_check_runs()
        passed_all_test = all(check_run.conclusion == 'success' for check_run in check_runs)

        writer.writerow([commit_id, commit_message, contributor, commit_date, changed_files, author_name, author_email, passed_all_test])

print("All files loaded successfully")